In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
#!pip install imbalanced-learn
#!pip3 install -U imbalanced-learn
from imblearn.over_sampling import SMOTE
import imblearn

In [117]:
loan_df = pd.read_csv("Resources/accepted2.csv")

In [118]:
loan_df1 = loan_df.drop(["mths_since_last_delinq","mths_since_last_record","dti_joint","annual_inc_joint","zip_code","addr_state","title"], axis=1)



In [119]:
loan_df2 = loan_df1.dropna()


In [288]:
loan_df3 = loan_df2[["recoveries","total_rec_late_fee","total_rec_prncp","delinq_2yrs","fico_range_high","loan_status","total_pymnt"]]




In [289]:
loan_df3["loan_status"].value_counts()

Fully Paid            39297
Current               29528
Charged Off            9788
Late (31-120 days)      702
In Grace Period         290
Late (16-30 days)       136
Default                   1
Name: loan_status, dtype: int64

In [290]:
loan_df4 = loan_df3.apply(lambda x: x.mask(x.map(x.value_counts())<= 9788, 'Late') if x.name=='loan_status' else x)

loan_df5 = loan_df4.apply(lambda x: x.mask(x.map(x.value_counts())>= 29528, 'Fully Paid') if x.name=='loan_status' else x)


In [291]:
loan_df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79742 entries, 0 to 106996
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   recoveries          79742 non-null  float64
 1   total_rec_late_fee  79742 non-null  float64
 2   total_rec_prncp     79742 non-null  float64
 3   delinq_2yrs         79742 non-null  float64
 4   fico_range_high     79742 non-null  int64  
 5   loan_status         79742 non-null  object 
 6   total_pymnt         79742 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 4.9+ MB


In [306]:
dummies_df = pd.get_dummies(loan_df5)

dummies_df.to_csv("status_predictor.csv")

# Logistic Regression

In [293]:
from sklearn.model_selection import train_test_split
# X = train_df
X = dummies_df.drop(["loan_status_Fully Paid","loan_status_Late"], axis=1)

y = dummies_df["loan_status_Fully Paid"]

oversample = SMOTE()

X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [294]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [295]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier

classifier.fit(X_train_scaled, y_train)

#logistic regression
print(f'Training Score: {classifier.score(X_train_scaled, y_train)}')
print(f'Testing Score: {classifier.score(X_test_scaled, y_test)}')

Training Score: 0.8400282844329068
Testing Score: 0.8435474965855927


In [296]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


y_true = y_test

y_pred = classifier.predict(X_test)

confusion_matrix(y_true, y_pred)

confusion_matrix(y_test, y_pred)

/Users/andrealynnbaker/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([[14202,  2848],
       [ 3431, 13932]])

In [297]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_true, y_pred)


0.817678888079499

In [298]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=["Late","Fully Paid"]))

              precision    recall  f1-score   support

        Late       0.81      0.83      0.82     17050
  Fully Paid       0.83      0.80      0.82     17363

    accuracy                           0.82     34413
   macro avg       0.82      0.82      0.82     34413
weighted avg       0.82      0.82      0.82     34413



In [299]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()

In [300]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],
    'weights': ['uniform', 'distance'],
    'leaf_size': [10, 50, 100, 500]
}
grid_clf = GridSearchCV(model, param_grid, verbose=3)

In [301]:
grid_clf.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV 1/5] END leaf_size=10, n_neighbors=1, weights=uniform;, score=0.908 total time=   0.8s
[CV 2/5] END leaf_size=10, n_neighbors=1, weights=uniform;, score=0.907 total time=   0.7s
[CV 3/5] END leaf_size=10, n_neighbors=1, weights=uniform;, score=0.910 total time=   0.8s
[CV 4/5] END leaf_size=10, n_neighbors=1, weights=uniform;, score=0.914 total time=   0.8s
[CV 5/5] END leaf_size=10, n_neighbors=1, weights=uniform;, score=0.913 total time=   0.9s
[CV 1/5] END leaf_size=10, n_neighbors=1, weights=distance;, score=0.908 total time=   0.2s
[CV 2/5] END leaf_size=10, n_neighbors=1, weights=distance;, score=0.907 total time=   0.2s
[CV 3/5] END leaf_size=10, n_neighbors=1, weights=distance;, score=0.910 total time=   0.2s
[CV 4/5] END leaf_size=10, n_neighbors=1, weights=distance;, score=0.914 total time=   0.2s
[CV 5/5] END leaf_size=10, n_neighbors=1, weights=distance;, score=0.913 total time=   0.2s
[CV 1/5] END leaf_size=

[CV 5/5] END leaf_size=10, n_neighbors=17, weights=distance;, score=0.894 total time=   0.4s
[CV 1/5] END leaf_size=10, n_neighbors=19, weights=uniform;, score=0.861 total time=   0.9s
[CV 2/5] END leaf_size=10, n_neighbors=19, weights=uniform;, score=0.859 total time=   1.0s
[CV 3/5] END leaf_size=10, n_neighbors=19, weights=uniform;, score=0.862 total time=   1.0s
[CV 4/5] END leaf_size=10, n_neighbors=19, weights=uniform;, score=0.866 total time=   0.9s
[CV 5/5] END leaf_size=10, n_neighbors=19, weights=uniform;, score=0.862 total time=   0.9s
[CV 1/5] END leaf_size=10, n_neighbors=19, weights=distance;, score=0.890 total time=   0.4s
[CV 2/5] END leaf_size=10, n_neighbors=19, weights=distance;, score=0.890 total time=   0.4s
[CV 3/5] END leaf_size=10, n_neighbors=19, weights=distance;, score=0.890 total time=   0.4s
[CV 4/5] END leaf_size=10, n_neighbors=19, weights=distance;, score=0.894 total time=   0.4s
[CV 5/5] END leaf_size=10, n_neighbors=19, weights=distance;, score=0.893 t

[CV 5/5] END leaf_size=50, n_neighbors=15, weights=distance;, score=0.895 total time=   0.4s
[CV 1/5] END leaf_size=50, n_neighbors=17, weights=uniform;, score=0.864 total time=   1.0s
[CV 2/5] END leaf_size=50, n_neighbors=17, weights=uniform;, score=0.861 total time=   0.9s
[CV 3/5] END leaf_size=50, n_neighbors=17, weights=uniform;, score=0.864 total time=   0.9s
[CV 4/5] END leaf_size=50, n_neighbors=17, weights=uniform;, score=0.868 total time=   1.0s
[CV 5/5] END leaf_size=50, n_neighbors=17, weights=uniform;, score=0.865 total time=   0.9s
[CV 1/5] END leaf_size=50, n_neighbors=17, weights=distance;, score=0.892 total time=   0.4s
[CV 2/5] END leaf_size=50, n_neighbors=17, weights=distance;, score=0.891 total time=   0.4s
[CV 3/5] END leaf_size=50, n_neighbors=17, weights=distance;, score=0.890 total time=   0.4s
[CV 4/5] END leaf_size=50, n_neighbors=17, weights=distance;, score=0.895 total time=   0.4s
[CV 5/5] END leaf_size=50, n_neighbors=17, weights=distance;, score=0.894 t

[CV 4/5] END leaf_size=100, n_neighbors=13, weights=distance;, score=0.898 total time=   0.7s
[CV 5/5] END leaf_size=100, n_neighbors=13, weights=distance;, score=0.896 total time=   0.5s
[CV 1/5] END leaf_size=100, n_neighbors=15, weights=uniform;, score=0.867 total time=   1.4s
[CV 2/5] END leaf_size=100, n_neighbors=15, weights=uniform;, score=0.865 total time=   1.3s
[CV 3/5] END leaf_size=100, n_neighbors=15, weights=uniform;, score=0.867 total time=   1.2s
[CV 4/5] END leaf_size=100, n_neighbors=15, weights=uniform;, score=0.869 total time=   1.1s
[CV 5/5] END leaf_size=100, n_neighbors=15, weights=uniform;, score=0.867 total time=   1.3s
[CV 1/5] END leaf_size=100, n_neighbors=15, weights=distance;, score=0.894 total time=   0.7s
[CV 2/5] END leaf_size=100, n_neighbors=15, weights=distance;, score=0.893 total time=   0.6s
[CV 3/5] END leaf_size=100, n_neighbors=15, weights=distance;, score=0.892 total time=   0.6s
[CV 4/5] END leaf_size=100, n_neighbors=15, weights=distance;, sc

[CV 3/5] END leaf_size=500, n_neighbors=11, weights=distance;, score=0.894 total time=   1.5s
[CV 4/5] END leaf_size=500, n_neighbors=11, weights=distance;, score=0.900 total time=   1.2s
[CV 5/5] END leaf_size=500, n_neighbors=11, weights=distance;, score=0.898 total time=   1.1s
[CV 1/5] END leaf_size=500, n_neighbors=13, weights=uniform;, score=0.869 total time=   1.5s
[CV 2/5] END leaf_size=500, n_neighbors=13, weights=uniform;, score=0.867 total time=   1.6s
[CV 3/5] END leaf_size=500, n_neighbors=13, weights=uniform;, score=0.869 total time=   2.0s
[CV 4/5] END leaf_size=500, n_neighbors=13, weights=uniform;, score=0.872 total time=   1.6s
[CV 5/5] END leaf_size=500, n_neighbors=13, weights=uniform;, score=0.870 total time=   1.6s
[CV 1/5] END leaf_size=500, n_neighbors=13, weights=distance;, score=0.895 total time=   1.1s
[CV 2/5] END leaf_size=500, n_neighbors=13, weights=distance;, score=0.893 total time=   1.1s
[CV 3/5] END leaf_size=500, n_neighbors=13, weights=distance;, sc

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'leaf_size': [10, 50, 100, 500],
                         'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],
                         'weights': ['uniform', 'distance']},
             verbose=3)

In [302]:
print(grid_clf.best_params_)

{'leaf_size': 10, 'n_neighbors': 1, 'weights': 'uniform'}


In [303]:
print(grid_clf.best_score_)

0.9103422769155893


In [304]:
y_true = y_test

y_pred = grid_clf.predict(X_test)

confusion_matrix(y_true, y_pred)

confusion_matrix(y_test, y_pred)

array([[16068,   982],
       [ 1837, 15526]])

In [305]:
import pickle

model = KNeighborsClassifier()
model.fit(X_train, y_train)
filename = 'loan_status_predictor.sav'
pickle.dump(model, open(filename, 'wb'))